<a href="https://colab.research.google.com/github/tommymmcguire/Duke-AI-MEng-chatbot/blob/main/notebooks/finetune_mistral_instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup Lit GPT

In [1]:
!pip install 'litgpt[all] @ git+https://github.com/Lightning-AI/litgpt'

  Cloning https://github.com/Lightning-AI/litgpt to /tmp/pip-install-cx81yzpa/litgpt_b9e5b57fed394e2db7f4d3976a262236
  Running command git clone --filter=blob:none --quiet https://github.com/Lightning-AI/litgpt /tmp/pip-install-cx81yzpa/litgpt_b9e5b57fed394e2db7f4d3976a262236
  Resolved https://github.com/Lightning-AI/litgpt to commit 3516beacd3ff2ccca51fdc6e2471ff5adcc35e4f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.2/192.2 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 98.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 67.0 MB/s eta 0:00:0

In [ ]:
!pwd

/content


### Download Model - Mistral7B

In [2]:
!litgpt download --repo_id mistralai/Mistral-7B-v0.1
# !litgpt convert_hf_checkpoint --checkpoint_dir checkpoints/mistralai/Mistral-7B

Setting HF_HUB_ENABLE_HF_TRANSFER=1
config.json: 100% 571/571 [00:00<00:00, 3.95MB/s]
generation_config.json: 100% 116/116 [00:00<00:00, 593kB/s]
pytorch_model-00001-of-00002.bin: 100% 9.94G/9.94G [00:30<00:00, 324MB/s]
pytorch_model-00002-of-00002.bin: 100% 5.06G/5.06G [00:16<00:00, 313MB/s]
pytorch_model.bin.index.json: 100% 23.9k/23.9k [00:00<00:00, 80.2MB/s]
tokenizer.json: 100% 1.80M/1.80M [00:00<00:00, 4.37MB/s]
tokenizer.model: 100% 493k/493k [00:00<00:00, 452MB/s]
tokenizer_config.json: 100% 967/967 [00:00<00:00, 5.64MB/s]
Converting checkpoint files to LitGPT format.
Processing checkpoints/mistralai/Mistral-7B-v0.1/pytorch_model-00001-of-00002.bin
Loading 'model.embed_tokens.weight' into RAM
Loading 'model.layers.0.self_attn.o_proj.weight' into RAM
Loading 'model.layers.0.mlp.gate_proj.weight' into RAM
Loading 'model.layers.0.mlp.up_proj.weight' into RAM
Loading 'model.layers.0.mlp.down_proj.weight' into RAM
Loading 'model.layers.0.input_layernorm.weight' into RAM
Loading 'mod

In [3]:
!litgpt generate base --precision bf16-true --checkpoint_dir checkpoints/mistralai/Mistral-7B-v0.1 --prompt "Are you good at answering Questions?"

Loading model 'checkpoints/mistralai/Mistral-7B-v0.1/lit_model.pth' with {'name': 'Mistral-7B-v0.1', 'hf_config': {'name': 'Mistral-7B-v0.1', 'org': 'mistralai'}, 'scale_embeddings': False, 'block_size': 4096, 'vocab_size': 32000, 'padding_multiple': 512, 'padded_vocab_size': 32000, 'n_layer': 32, 'n_head': 32, 'head_size': 128, 'n_embd': 4096, 'rotary_percentage': 1.0, 'parallel_residual': False, 'bias': False, 'lm_head_bias': False, 'n_query_groups': 8, 'shared_attention_norm': False, 'norm_class_name': 'RMSNorm', 'norm_eps': 1e-05, 'mlp_class_name': 'LLaMAMLP', 'gelu_approximate': 'none', 'intermediate_size': 14336, 'rope_condense_ratio': 1, 'rope_base': 10000, 'n_expert': 0, 'n_expert_per_token': 0, 'rope_n_elem': 128}
Time to instantiate model: 0.32 seconds.
Time to load the model weights: 12.37 seconds.
Seed set to 1234
Are you good at answering Questions? Have you ever wondered how you would do in a Q&A? In this post, I will give you 10 Q&A Questions for you to practice with and

## Get Data

## Finetune Model

In [ ]:
!litgpt finetune lora \
  --checkpoint_dir "checkpoints/mistralai/Mistral-7B-v0.1/" \
  --precision bf16-true \
  --quantize bnb.fp4 \
  --data AlpacaGPT4\
  --out_dir "out/lora_weights/mistral7B-finetuned-test/" \
  --train.log_interval 16 \
  --train.save_interval 1 \
  --train.micro_batch_size 2 \
  --train.global_batch_size 32 \
  --train.max_seq_length 512 \
  --train.epochs 3 \
  --train.lr_warmup_steps=100

{'checkpoint_dir': PosixPath('checkpoints/mistralai/Mistral-7B-v0.1'),
 'data': AlpacaGPT4(mask_prompt=False,
                    val_split_fraction=0.03847,
                    prompt_style=<litgpt.prompts.Alpaca object at 0x79a72fa71a80>,
                    ignore_index=-100,
                    seed=42,
                    num_workers=4,
                    download_dir=PosixPath('data/alpacagpt4')),
 'devices': 1,
 'eval': EvalArgs(interval=100, max_new_tokens=100, max_iters=100),
 'logger_name': 'csv',
 'lora_alpha': 16,
 'lora_dropout': 0.05,
 'lora_head': False,
 'lora_key': False,
 'lora_mlp': False,
 'lora_projection': False,
 'lora_query': True,
 'lora_r': 8,
 'lora_value': True,
 'out_dir': PosixPath('out/lora_weights/mistral7B-finetuned-test'),
 'precision': 'bf16-true',
 'quantize': 'bnb.fp4',
 'seed': 1337,
 'train': TrainArgs(save_interval=1,
                    log_interval=16,
                    global_batch_size=32,
                    micro_batch_size=2,
          

In [17]:
!litgpt generate base \
  --checkpoint_dir "checkpoints/instruct-tuned" \
  --prompt f"{prompt_RAG}" \
  --precision bf16-true

Loading model 'checkpoints/instruct-tuned/lit_model.pth' with {'name': 'Mistral-7B-v0.1', 'hf_config': {'name': 'Mistral-7B-v0.1', 'org': 'mistralai'}, 'scale_embeddings': False, 'block_size': 4096, 'vocab_size': 32000, 'padding_multiple': 512, 'padded_vocab_size': 32000, 'n_layer': 32, 'n_head': 32, 'head_size': 128, 'n_embd': 4096, 'rotary_percentage': 1.0, 'parallel_residual': False, 'bias': False, 'lm_head_bias': False, 'n_query_groups': 8, 'shared_attention_norm': False, 'norm_class_name': 'RMSNorm', 'norm_eps': 1e-05, 'mlp_class_name': 'LLaMAMLP', 'gelu_approximate': 'none', 'intermediate_size': 14336, 'rope_condense_ratio': 1, 'rope_base': 10000, 'n_expert': 0, 'n_expert_per_token': 0, 'rope_n_elem': 128}
Time to instantiate model: 0.15 seconds.
Time to load the model weights: 12.26 seconds.
Seed set to 1234
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
fCHUNK: FLEXIBILITY AND OPTIONS 4+1: BSE+Master's

In [15]:
prompt_RAG = "CHUNK: FLEXIBILITY AND OPTIONS \
4+1: BSE+Master's Option for Duke Undergraduates \
Duke undergraduate students can complete undergrad and this master's degree in just five (5) years. \
Scholarship opportunity: The AI 4+1 BSE+Master's scholarship covers 20 percent of the costs. Eligibility and other conditions apply. \
MD+MEng in Artificial Intelligence Dual Degree \
Medical students at Duke can complete this degree during their third year. See Duke MEDx website and School of Medicine bulletin for details.\
Scholarship opportunity: The MD+MEng AI scholarship covers 20 percent of the costs. Eligibility and other conditions apply.\
=== \
CURRICULUM SCHEDULES \
The core of the curriculum follows a cohort-based course sequence \
On-Campus Accelerated Option: 12 Months\
Summer:\
Pre-requisite \
AIPI 503: Python & Data Science Math Bootcamp \
Fall: \
AIPI 510: Sourcing Data for Analytics \
AIPI 520: Modeling Process & Algorithms \
AIPI Departmental Elective \
MENG 570: Business Fundamentals for Engineers \
AIPI 501: Industry Seminar Series \
EGR 590: Career Strategy & Design \
====== \
prompt: Is is possible to complete the Meng AI program in just 1 more year after undergrad and what are prerequisites?"

### Merging LoRA Weights

In [3]:
import os
os.mkdir("checkpoints/instruct-tuned")
os.mkdir("checkpoints/instruct-tuned-hf")

In [4]:
!unzip drive/MyDrive/instruct-alpaca.zip -d checkpoints/instruct-tuned

Archive:  drive/MyDrive/instruct-alpaca.zip
  inflating: checkpoints/instruct-tuned/lit_model.pth.lora  
  inflating: checkpoints/instruct-tuned/tokenizer_config.json  
  inflating: checkpoints/instruct-tuned/model_config.yaml  
  inflating: checkpoints/instruct-tuned/tokenizer.json  
  inflating: checkpoints/instruct-tuned/generation_config.json  
  inflating: checkpoints/instruct-tuned/hyperparameters.yaml  
 extracting: checkpoints/instruct-tuned/prompt_style.yaml  
  inflating: checkpoints/instruct-tuned/tokenizer.model  
   creating: checkpoints/instruct-tuned/.ipynb_checkpoints/


In [5]:
!litgpt merge_lora \
 --checkpoint_dir "checkpoints/instruct-tuned" \
 --pretrained_checkpoint_dir "checkpoints/mistralai/Mistral-7B-v0.1" \
 --precision bf16-true

Saved merged weights to 'checkpoints/instruct-tuned/lit_model.pth'


### Zip and Save

In [ ]:
import os
os.chdir("/content/checkpoints/")
!zip -r Mistral-instruct-alpaca-400.zip Mistral-instruct-alpaca-400

  adding: Mistral-instruct-alpaca-400/ (stored 0%)
  adding: Mistral-instruct-alpaca-400/lit_model.pth.lora (deflated 21%)
  adding: Mistral-instruct-alpaca-400/lit_model.pth


zip error: Interrupted (aborting)


In [ ]:
!zip -r mistral7B-finetuned-alpaca-final.zip out/lora_weights/mistral7B-finetuned-

  adding: out/lora_weights/mistral7B-finetuned-test/step-000001/ (stored 0%)
  adding: out/lora_weights/mistral7B-finetuned-test/step-000001/lit_model.pth.lora (deflated 52%)
  adding: out/lora_weights/mistral7B-finetuned-test/step-000001/tokenizer_config.json (deflated 68%)
  adding: out/lora_weights/mistral7B-finetuned-test/step-000001/model_config.yaml (deflated 42%)
  adding: out/lora_weights/mistral7B-finetuned-test/step-000001/tokenizer.json (deflated 74%)
  adding: out/lora_weights/mistral7B-finetuned-test/step-000001/generation_config.json (deflated 22%)
  adding: out/lora_weights/mistral7B-finetuned-test/step-000001/hyperparameters.yaml (deflated 46%)
  adding: out/lora_weights/mistral7B-finetuned-test/step-000001/prompt_style.yaml (stored 0%)
  adding: out/lora_weights/mistral7B-finetuned-test/step-000001/tokenizer.model (deflated 55%)


In [ ]:
!cp mistral7B-finetuned-alpaca-final.zip /content/drive/MyDrive/

cp: cannot stat 'mistral7B-finetuned-logic-final.zip': No such file or directory


In [ ]:
!unzip instruct-alpaca.zip -d checkpoints/Mistral-instruct-alpaca

Archive:  instruct-alpaca.zip
  inflating: checkpoints/Mistral-instruct-alpaca/lit_model.pth.lora  
  inflating: checkpoints/Mistral-instruct-alpaca/tokenizer_config.json  
  inflating: checkpoints/Mistral-instruct-alpaca/model_config.yaml  
  inflating: checkpoints/Mistral-instruct-alpaca/tokenizer.json  
  inflating: checkpoints/Mistral-instruct-alpaca/generation_config.json  
  inflating: checkpoints/Mistral-instruct-alpaca/hyperparameters.yaml  
 extracting: checkpoints/Mistral-instruct-alpaca/prompt_style.yaml  
  inflating: checkpoints/Mistral-instruct-alpaca/tokenizer.model  
   creating: checkpoints/Mistral-instruct-alpaca/.ipynb_checkpoints/


### Convert to Huggingface

In [6]:
!litgpt convert from_litgpt \
    --checkpoint_dir checkpoints/instruct-tuned \
    --output_dir checkpoints/instruct-tuned-hf

Loading 'lm_head.weight' into RAM
Loading 'transformer.wte.weight' into RAM
Loading 'transformer.h.0.norm_1.weight' into RAM
Loading 'transformer.h.0.attn.attn.weight' into RAM
Loading 'transformer.h.0.attn.proj.weight' into RAM
Loading 'transformer.h.0.norm_2.weight' into RAM
Loading 'transformer.h.0.mlp.fc_1.weight' into RAM
Loading 'transformer.h.0.mlp.fc_2.weight' into RAM
Loading 'transformer.h.0.mlp.proj.weight' into RAM
Loading 'transformer.h.1.norm_1.weight' into RAM
Loading 'transformer.h.1.attn.attn.weight' into RAM
Loading 'transformer.h.1.attn.proj.weight' into RAM
Loading 'transformer.h.1.norm_2.weight' into RAM
Loading 'transformer.h.1.mlp.fc_1.weight' into RAM
Loading 'transformer.h.1.mlp.fc_2.weight' into RAM
Loading 'transformer.h.1.mlp.proj.weight' into RAM
Loading 'transformer.h.2.norm_1.weight' into RAM
Loading 'transformer.h.2.attn.attn.weight' into RAM
Loading 'transformer.h.2.attn.proj.weight' into RAM
Loading 'transformer.h.2.norm_2.weight' into RAM
Loading 'tra

In [7]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
from transformers import AutoModel

state_dict = torch.load('checkpoints/instruct-tuned-hf/model.pth')
model = AutoModel.from_pretrained("mistralai/Mistral-7B-v0.1", state_dict=state_dict)

In [1]:
model.save_pretrained("checkpoints/custom-instruct-mistral7B")

NameError: name 'model' is not defined

In [1]:
model.push_to_hub("mkeohane01/mistral-instruct-duke590")

NameError: name 'model' is not defined

In [ ]:
from transformers import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="checkpoints/custom-instruct-mistral7B", tokenizer=tokenizer, hub_model_id="mkeohane01/mistral-instruct-duke590"
)